## install libraries

In [1]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## library import

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
import torch
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

GLOBAL_SEED = 42

## load dataset

In [3]:
dataset = load_dataset("persiannlp/parsinlu_query_paraphrasing")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1830 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/898 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1830
    })
    test: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1916
    })
    validation: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 898
    })
})

In [4]:
dataset["test"]

Dataset({
    features: ['q1', 'q2', 'category', 'label'],
    num_rows: 1916
})

In [5]:
train_df = pd.DataFrame(dataset["train"]) # for examples in one-shot and few-shot learning
test_df = pd.DataFrame(dataset["test"]) # for evaluation
test_df

,q1,q2,category,label
0,آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنتر...,چه چیزی روح فرد را می شکند؟,qqp,0
1,چگونه می توانم تماشای فیلم های پورنو را متوقف ...,برای جلوگیری از تماشای کامل پورنو باید چه کاری...,qqp,1
2,چه کسانی امام علی را خدا میدانند؟,چه کسانی می توانند امام زمان را ببینند؟,natural,0
3,آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایال...,شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا...,qqp,1
4,چگونه می توانم سوالی را در این باره بپرسم؟,چگونه می توانم سوال بپرسم؟,qqp,0
...,...,...,...,...
1911,چه چیزهایی روزه زن را باطل میکند اهل سنت؟,چه چیزهایی روزه زن را باطل میکند مکارم شیرازی؟,natural,0
1912,وای چه پسری وای عجب تاج سری؟,دانلود اهنگ پسر وای وای چه پسری وای عجب تاج سری؟,natural,1
1913,از چه زمانی حرکات جنین کم میشود؟,از چه زمانی حرکات جنین حس میشود؟,natural,0
1914,کدام یک از شرکتها دارای رشد شغلی بهتری دارند؟ ...,۴ لوله ورودی و ۲ لوله خروجی ، ورودی آب را پیدا...,qqp,0


## test model on a single example

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
torch.random.manual_seed(GLOBAL_SEED)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
!ls "/root/.cache/huggingface/hub/"

models--microsoft--Phi-3-mini-4k-instruct  version.txt


In [9]:
!rm -rf "/root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct"

In [10]:
messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
pipe

In [11]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": None,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

 To solve the equation 2x + 3 = 7, follow these steps:

1. Subtract 3 from both sides of the equation: 2x + 3 - 3 = 7 - 3
2. Simplify: 2x = 4
3. Divide both sides by 2: 2x/2 = 4/2
4. Simplify: x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


In [12]:
template = "Answer the questions in a single word"
prompt = f"در ادامه دو سوال آمده است. \
            آیا این دو سوال مترادف هستند یا خیر؟ \
            اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. \
            اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. \n\
            سوال اول: {train_df.iloc[100]['q1']} \n\
            سوال دوم: {train_df.iloc[100]['q2']}"

messages = [
    # {"role": "system", "content": template},
    {"role": "user", "content": prompt},
]


generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": None,
    "do_sample": False,
}
# terminators = [
#     pipe.tokenizer.eos_token_id,
#     pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]
# generation_args = {
#     "max_new_tokens": 256,
#     "return_full_text": False,
#     "temperature": 0.10,
#     "do_sample": True,
#     "top_p": 0.95,
#     # "eos_token_id": terminators,
# }

output = pipe(messages, **generation_args)

label_mapping = {
    0: "خیر",
    1: "بله"
}

print(f"Real label is: {label_mapping[int(train_df.iloc[100]['label'])]}")
model_output = output[0]['generated_text']
print(f"Generated text: {model_output}")

Real label is: خیر
Generated text:  سوال اول: بفهمیم که بیهوشی حساسیت داریم ترجمه ای است که "ما داریم حساسیت بیهوشی داریم" است.

سوال دوم: بفهمیم که گلوتن حساسیت داریم ترجمه ای است که "ما داریم حساسیت به گلوتن داریم" است.


این دو سوالات دارای مترادفی نیستند و به کلمه جواب "خیر" یا "بله" تولید کنید.


## inference on different scenarios

In [13]:
def create_prompt(scenario, q1, q2, examples=None):
    if scenario == "Zero-shot":
        return f"در ادامه دو سوال آمده است. \
                آیا این دو سوال مترادف هستند یا خیر؟ \
                اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. \
                اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. \n\
                سوال اول: {q1} \n\
                سوال دوم: {q2}"
    elif scenario == "One-shot":
        return f"در ادامه دو سوال آمده است. \
                آیا این دو سوال مترادف هستند یا خیر؟ \
                اگر مترادف هستند فقط در یک کلمه 'بله' را تولید کنید. \
                اگر مترادف نیستند فقط در یک کلمه 'خیر' را تولید کنید. \n\
                مثال: \n\
                سوال اول: {examples.iloc[0]['q1']} \n\
                سوال دوم: {examples.iloc[0]['q2']} \n\
                جواب: {label_mapping[int(examples.iloc[0]['label'])]}\n\
                حال با دیدن مثال جواب دهید. \n\
                سوال اول: {q1} \n\
                سوال دوم: {q2}"
    elif scenario == "Five-shot":
        examples_text = ""
        for i, example in examples.iterrows():
            examples_text += f"سوال اول: {example['q1']} \n\
                               سوال دوم: {example['q2']} \n\
                               جواب: {label_mapping[int(example['label'])]}\n\n"
        return f"در ادامه دو سوال آمده است. \
                آیا این دو سوال مترادف هستند یا خیر؟ \
                اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. \
                اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. \n\
                مثال ها: \n\
                {examples_text} \
                حال با دیدن مثال ها جواب دهید. \n\
                سوال اول: {q1} \n\
                سوال دوم: {q2}"

In [14]:
def generate_output_from_prompt(prompt, max_new_tokens=500):
    messages = [
        {"role": "user", "content": prompt},
    ]

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "return_full_text": False,
        "temperature": None,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)

    return output[0]['generated_text']

In [15]:
def find_first_label(labels, text):
    # Initialize a dictionary to store the position of each label
    positions = {label: text.find(label) for label in labels}

    # Filter out labels that are not found (position -1)
    valid_positions = {label: pos for label, pos in positions.items() if pos != -1}

    # If no valid positions are found, return None
    if not valid_positions:
        return None

    # Find the label with the minimum position
    first_label = min(valid_positions, key=valid_positions.get)

    return first_label

In [16]:
def test_scenario(scenario, test_df, log_df, examples=None):
    correct = 0
    total = len(test_df)
    labels = ["بله", "خیر"]

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc=f"Running {scenario} scenario..."):
        prompt = create_prompt(scenario, row['q1'], row['q2'], examples)
        output = generate_output_from_prompt(prompt)
        label = int(row['label'])
        if label == 1:
            # if output == "بله":
            # if "بله" in output.split(" ")[0]:
            # if "بله" in output:
            if "بله" == find_first_label(labels, output):
                correct += 1
        else:
            # if output == "خیر":
            # if "خیر" in output.split(" ")[0]:
            # if "خیر" in output:
            if "بله" == find_first_label(labels, output):
                correct += 1
        log_df.loc[len(log_df)] = [row['q1'], row['q2'], output, label_mapping[label], scenario]

    return correct / total

In [19]:
log_df = pd.DataFrame(columns=["q1", "q2", "generated_text", "real_label", "scenario"])

scenarios = ["Zero-shot", "One-shot", "Five-shot"]

test_subset_examples = test_df.sample(n=10, random_state=GLOBAL_SEED) # because it takes too much to test all examples we use subset of it
one_shot_subset_examples = train_df.sample(n=5, random_state=GLOBAL_SEED)
# five_shot_subset_examples = train_df.sample(n=5, random_state=GLOBAL_SEED)
# balance between 0 and 1
grouped = train_df.groupby('label')
five_shot_subset_examples = grouped.apply(lambda x: x.sample(n=2 if x.name == '0' else 3, random_state=GLOBAL_SEED))
five_shot_subset_examples = five_shot_subset_examples.reset_index(drop=True)


for scenario in scenarios:
    if scenario == "Zero-shot":
        zero_shot_result = test_scenario(scenario, test_subset_examples, log_df,examples=None)
    elif scenario == "One-shot":
        one_shot_result = test_scenario(scenario, test_subset_examples, log_df, examples=one_shot_subset_examples)
    elif scenario == "Five-shot":
        five_shot_result = test_scenario(scenario, test_subset_examples, log_df, examples=five_shot_subset_examples)
    else:
        # exception
        raise ValueError("Invalid scenario")

model_id = "Phi-3-mini-4k-instruct"
print("Experiment result:")
print(f"Model Name: {model_id}")
print(f"Zero-shot accuracy: {zero_shot_result}")
print(f"One-shot accuracy: {one_shot_result}")
print(f"Five-shot accuracy: {five_shot_result}")

Running Five-shot scenario...: 100%|██████████| 10/10 [03:45<00:00, 22.56s/it]

Experiment result:
Model Name: Phi-3-mini-4k-instruct
Zero-shot accuracy: 0.4
One-shot accuracy: 0.9
Five-shot accuracy: 0.7


In [20]:
# save log_df
model_name = "Phi-3-mini-4k-instruct"
log_df.to_csv(f"log_df_{model_name}.csv", index=False, encoding="utf-8-sig")

In [21]:
log_df

,q1,q2,generated_text,real_label,scenario
0,ويتامين ب در چه چيزهايي وجود دارد؟,ويتامين b12 در چه چيزهايي وجود دارد؟,سوال اول: ويتامين ب در چه چيزهايي وجود دارد؟\...,خیر,Zero-shot
1,داستان معراج پیامبر در کدام سوره است؟,داستان معراج پیامبر در کدام سوره قرانی امده است؟,این دو سوال مترادف نیستند. سوال اول: داستان م...,بله,Zero-shot
2,کارتهای اعتباری در مشاغل کوچک و رستوران های ما...,کارتهای اعتباری در مشاغل کوچک و رستورانهای اسر...,این دو سوال مترادف نیستند. کارتهای اعتباری در...,خیر,Zero-shot
3,زیرنویس فارسی فیلم دره من چه سرسبز بود؟,دانلود زیرنویس فارسی فیلم دره من چه سرسبز بود؟,"این دو سوال مترادف است.\n\nکلمه جواب ""بله"" با...",بله,Zero-shot
4,از کجا بفهمیم باتری لپ تاپ سالم است؟,از کجا بفهمیم باتری لپ تاپ خراب است؟,سوال اول: از کجا بفهمیم باتری لپ تاپ سالم است...,خیر,Zero-shot
5,فلات ایران شبیه کدام شکل هندسی است؟,کدام پرچم شبیه پرچم ایران است؟,سوال اول: فلات ایران شبیه کدام شکل هندسی است؟...,خیر,Zero-shot
6,خانواده ویتامین b در چه چیزهایی وجود دارد؟,ویتامین گروه b در چه مواد غذایی وجود دارد؟,سوال اول: خانواده ویتامین b در چه چیزهایی وجو...,بله,Zero-shot
7,برخی از ویژگی های انقباضات غیر عادی و متمرکز چ...,علائم انقباضات غیر عادی و متمرکز چیست؟,سوال اول: برخی از ویژگی‌های انقباضات غیر عادی...,بله,Zero-shot
8,سود سهام عدالت شامل چه کسانی می شود؟,سهام عدالت شامل حال چه کسانی می شود؟,"سوال اول: سود سهام عدالت شامل کلمه ""ملایم"" اس...",خیر,Zero-shot
9,تست بارداری به غیر از بیبی چک؟,چه موقع میشه از تست بارداری استفاده کرد؟,سوال اول: تست بارداری به غیر از بیبی چک؟ - خی...,خیر,Zero-shot


In [22]:
# save the printed result in a csv file
results = {
    "Model Name": [model_id],
    "Zero-shot accuracy": [zero_shot_result],
    "One-shot accuracy": [one_shot_result],
    "Five-shot accuracy": [five_shot_result]
}

df = pd.DataFrame(results)

df.to_csv(f"experiment_results_{model_name}.csv", index=False)

In [23]:
df

,Model Name,Zero-shot accuracy,One-shot accuracy,Five-shot accuracy
0,Phi-3-mini-4k-instruct,0.4,0.9,0.7
